# 1. 장르값 유사도 기반 추천시스템

## 데이터 불러오기 - TDMB 5000 영화 데이터 세트

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

C:\Users\taeeu\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\taeeu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\Users\taeeu\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)
#(4803, 20)->4803개의 레코드와 20개의 피처로 구성되어있음

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [3]:
#CB를 수행하는데 필요한 컬럼만 남김
movies_df=movies[['id','title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [4]:
movies_df[['genres','keywords']]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
...,...,...
4798,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi..."
4799,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[]
4800,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam..."
4801,[],[]


In [5]:
pd.set_option('max_colwidth', 100)
#행 한개만 출력해봄
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [6]:
#컬럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출
from ast import literal_eval

In [7]:
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)

In [8]:
#genres, keywords 컬럼은 문자열이 아닌 리스트 내부에 여러 장르 딕셔너리로 구성된 객체가됨
#컬럼에서 ['Action']/['Adventure']과 같은 장르명만 리스트의 객체로 추출
movies_df['genres']=movies_df['genres'].apply(lambda x : [y['name'] for y in x])

In [9]:
movies_df['keywords']=movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


In [10]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


## 장르값의 CountVectorizer

In [11]:
#CB : 장르값으로 유사도를 비교한 뒤 높은 평점을 갖는 영화를 추천
#genres컬럼을 문자열로 변경한 뒤 CountVectorizer로 피처 벡터화한 행렬값에 코사인 유사도를 적용해 유사성 판단
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
#CountVectorizer를 적용하기 위해 공백문자로 word단위가 구분되는 문자열로 변환
movies_df['genres_literal']=movies_df['genres'].apply(lambda x : (' ').join(x))

In [13]:
#변환 확인
print(movies_df[['genres']][:1])
print(movies_df[['genres_literal']][:1])

                                          genres
0  [Action, Adventure, Fantasy, Science Fiction]
                             genres_literal
0  Action Adventure Fantasy Science Fiction


In [14]:
#CountVectorizer함수->count_vect이라는 함수명으로 생성
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
#min_df : 단어장에 포함되기 위한 최소빈도

#'genres_literal'컬럼으로 CountVector생성
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)
#4803개 컬럼이었으므로 4803개 레코드, 276개의 개별단어 피처로 구성된 피처벡터 행렬 생성

(4803, 276)


## 장르값의 유사도

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)

#2개 행만 확인해보기!
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [16]:
# 유사도 값이 높은 순으로 위치 인덱스 추출(?)
genre_sim_sorted_ind=genre_sim.argsort()[:, ::-1]
## ::-1은 내림차순이라는 의미

print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


## 장르 유사도에 따라 영화를 추천

In [21]:
# 장르 유사도에 따라 영화를 추천하는 함수 find_sim_movie()생성
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    #인자로 입력된 movies_df(데이터프레임)에서 입력받은 'title'(제목) 컬럼이 입력된 값만 추출하여 저장
    title_movie = df[df['title'] == title_name]
    
    #title_named를 가진 데이터 프레임의 index 객체를 ndarray로 변환 -> 몇번째 영화인지? 인덱스 저장
    #sorted_ind(유사도값) 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index  추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    #추출된 top_n index를 출력. top_n index는 2차원 데이터
    print(similar_indexes)
    #데이터 프레임에서 index로 사용하기 위해 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
    
    #원래 df중에 인덱스에 포함된 행을 return
    return df.iloc[similar_indexes]

In [22]:
# find_sim_movie() 함수를 사용해 영화 '대부'와 장르별로 유사한 영화 10개를 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


# 2. 단점 개선
### - 좀 더 많은 후보군 선정, 평점에 따라 필터링

In [25]:
# vote_average : 평점 정보 사용
# 소수관객이 특정 영화에 만점이나 높은 평점을 부여해 왜곡된 데이터를 가짐
movies_df[['title', 'vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]
# 평점이 높은 순서대로(내림차순) 정렬

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [26]:
#왜곡된 평점 데이터를 회피하기 위해 가중치가 부여된 평점 사용..!

#C : 전체 영화에 대한 평균 평점
C=movies_df['vote_average'].mean()

#m : 평점을 부여하기 위한 최소 투표 횟수
#m값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점을 부여
#m값은 상위 60퍼센트에 해당하는 횟수를 기준
m=movies_df['vote_count'].quantile(0.6)

print('C :', round(C,3), 'm: ', round(m,3))

C : 6.092 m:  370.2


## 기존 평점을 가중 평점으로 변경

In [28]:
def weighted_vote_average(record):
    v=record['vote_count']
    R=record['vote_average']
    
    return ((v/(v+m))*R)+((m/(m+v))*C)

In [29]:
movies_df['weighted_vote']=movies_df.apply(weighted_vote_average, axis=1)

In [30]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


## 장르 유사성이 높은 영화 top_n의 2배수 -> weighted_vote값이 높은 순으로 추출

In [31]:
#가중치평점이 포함된 새로운 함수 정의
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie=df[df['title']==title_name]
    title_index=title_movie.index.values
    
    #top_n의 2배에 해당하는 장르 유사성이 높은 index 추출
    similar_indexes=sorted_ind[title_index, :(top_n*2)]
    similar_indexes=similar_indexes.reshape(-1)
    #기준 영화 index는 제외(자기자신 제외(?))
    similar_indexes=similar_indexes[similar_indexes !=title_index]
    
    #top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [32]:
similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]
#아까보다 좋은 추천결과 제공

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
